In [1]:
import numpy as np

from src.vrp_study.data_loader import parse_data
from vrp_study.managers.pdptw_routing_manager_builder import PDRoutingManagerBuilder
from vrp_study.or_tools.model import *
from vrp_study.initial_solution.pdptw_solution_builder import SolutionBuilder


In [2]:
benchmark_type = 'pdp_400'
name = 'LC1_4_9.txt'

In [3]:
depo, cargos, tariff = parse_data(f'../data/Li & Lim benchmark/benchmarks/{benchmark_type}/{name}')

In [4]:
p2coordinates = {}
p2coordinates.update({
    crg.nodes[i].id: crg.nodes[i].coordinates for crg in cargos for i in range(2)
})
p2coordinates[depo.id] = depo.coordinates
distance_matrix = {(u, v): np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2) for u, du in
                   p2coordinates.items() for
                   v, dv in p2coordinates.items()}
time_matrix = {(u, v): np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2) for u, du in p2coordinates.items() for
               v, dv in p2coordinates.items()}

In [5]:
len(cargos)

209

In [6]:
routing_manager = PDRoutingManagerBuilder(
    distance_matrix=distance_matrix,
    time_matrix=time_matrix,
).with_tariffs(tariff).with_cargos(cargos).with_depo(depo).build()

In [7]:
def build_model_factory() -> VRPModelFactory:
    return VRPModelFactory().with_dimensions(
        DistanceDimensionBuilderOrTools(),
        TimeDimensionBuilderOrTools(
            max_slack=max(node.end_time for node in routing_manager.nodes()),
            max_value=max(node.end_time for node in routing_manager.nodes())
        ),
        CapacityDimensionBuilderOrTools()
    ).with_constraints(
        TimeConstraintBuilderOrTools(),
        PDPConstraintBuilderOrTools()
    ).with_cost(
        LinearCostByKmBuilderOrTools()
    )


model_factory = build_model_factory()

In [8]:
callback = SolutionCallback()

In [9]:

initial_solution_builder = SolutionBuilder(
    solver=VRPSolver(
        model_config=ModelConfig(max_execution_time_minutes=1, max_solution_number=50)
    ),
    model_factory = model_factory
)

In [10]:
solver = VRPSolver(
    model_config=ModelConfig(max_execution_time_minutes=10),
    initial_solution_builder= initial_solution_builder,
    callback=callback
)

In [ ]:
solver.solve(
    model_factory.build_model(routing_manager)
)

2025-08-15 16:51:03.827 | INFO     | vrp_study.initial_solution.pdptw_solution_builder:get_initial_solution:131 - (1, 32.074999999999996)
2025-08-15 16:51:03.869 | INFO     | vrp_study.initial_solution.pdptw_solution_builder:get_initial_solution:131 - (1, 16.0875)
2025-08-15 16:51:03.925 | INFO     | vrp_study.initial_solution.pdptw_solution_builder:get_initial_solution:131 - (1, 8.09375)
2025-08-15 16:51:03.984 | INFO     | vrp_study.initial_solution.pdptw_solution_builder:get_initial_solution:131 - (1, 4.096875)
2025-08-15 16:51:04.028 | INFO     | vrp_study.initial_solution.pdptw_solution_builder:get_initial_solution:131 - (1, 2.0984374999999997)
2025-08-15 16:51:04.166 | INFO     | vrp_study.initial_solution.pdptw_solution_builder:get_initial_solution:131 - (5, 1.09921875)
2025-08-15 16:51:04.281 | INFO     | vrp_study.initial_solution.pdptw_solution_builder:get_initial_solution:131 - (14, 0.599609375)
2025-08-15 16:51:04.341 | INFO     | vrp_study.initial_solution.pdptw_solution_b

Assignment(distance0 (0) | distance1 (113) | distance2 (124) | distance3 (97) | distance4 (106) | distance5 (75) | distance6 (87) | distance7 (131) | distance8 (131) | distance9 (116) | distance10 (123) | distance11 (95) | distance12 (111) | distance13 (28) | distance14 (30) | distance15 (81) | distance16 (81) | distance17 (76) | distance18 (90) | distance19 (28) | distance20 (56) | distance21 (81) | distance22 (108) | distance23 (72) | distance24 (80) | distance25 (30) | distance26 (99) | distance27 (108) | distance28 (113) | distance29 (33) | distance30 (101) | distance31 (95) | distance32 (111) | distance33 (119) | distance34 (128) | distance35 (100) | distance36 (161) | distance37 (97) | distance38 (150) | distance39 (53) | distance40 (99) | distance41 (83) | distance42 (88) | distance43 (97) | distance44 (116) | distance45 (65) | distance46 (83) | distance47 (96) | distance48 (100) | distance49 (39) | distance50 (240) | distance51 (108) | distance52 (111) | distance53 (102) | dist

2025-08-15 16:51:06.982 | DEBUG    | vrp_study.or_tools.callback:__call__:36 - time: 5.634; new solution (8): 1001480; best solution: 1001480
2025-08-15 16:51:07.000 | DEBUG    | vrp_study.or_tools.callback:__call__:36 - time: 5.651; new solution (9): 1001331; best solution: 1001331
2025-08-15 16:51:07.015 | DEBUG    | vrp_study.or_tools.callback:__call__:36 - time: 5.667; new solution (10): 1001046; best solution: 1001046
2025-08-15 16:51:07.031 | DEBUG    | vrp_study.or_tools.callback:__call__:36 - time: 5.682; new solution (11): 1001039; best solution: 1001039
2025-08-15 16:51:07.045 | DEBUG    | vrp_study.or_tools.callback:__call__:36 - time: 5.696; new solution (12): 1000994; best solution: 1000994
2025-08-15 16:51:07.061 | DEBUG    | vrp_study.or_tools.callback:__call__:36 - time: 5.712; new solution (13): 994350; best solution: 994350
2025-08-15 16:51:07.126 | DEBUG    | vrp_study.or_tools.callback:__call__:36 - time: 5.778; new solution (14): 993009; best solution: 993009
2025-

In [ ]:
total_l = 0
for s in sol[0]:
    if len(s) > 0:
        # print(s)
        # a = routing_manager.nodes()[s[i]]
        # b = routing_manager.nodes()[s[i+1]]

        l = sum(routing_manager.get_distance(routing_manager.nodes()[s[i]], routing_manager.nodes()[s[i + 1]]) for i in
                range(len(s) - 1))
        # print(s, l)
        total_l += l
total_l